In [ ]:
import collections
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt
import shutil

#this cell converts pdfs to text
#if the pdf is a scanned image and needs its text extracted, it moves it to a new directory to be extracted in a new function


#moves files that need to be extracted with tesseract into a new directory. enter your own directory here
def moveImgFile(fname):
    shutil.move(fname, "/Users/samuelburgess/Documents/Jupyter/ARRegulations/pdf_textExtract2")

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    print("text")
    #print(text)
    if " " not in text:
        print("nottext")
        moveImgFile(fname)
    return text

#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir.
#Repeatedly calls the first 'convert' function over each document in the directory
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        if fileExtension == "pdf":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + pdf + ".txt"
            if not os.path.isfile(textFilename): 
                textFile = open(textFilename, "w") #make text file
                textFile.write(text) #write text to text file
pdfDir = "ARReg2/"
txtDir = "txt2/"
convertMultiple(pdfDir, txtDir)
   

In [ ]:
#if the pdf to text converter doesn't extract any text, then we can use this function to extract the text using
#pytesseract

#all of the files that were shuttled to a new directory in the previous cell will have to be converted to a
#.tiff file to proceed in this cell

try:
    from PIL import Image, ImageSequence
except ImportError:
    import Image
import pytesseract
from io import StringIO
import os
import glob
import sys, getopt
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

directory = 'tiff_textExtract2/*.tiff' #path
txtDir = 'txt2'
jpgFiles = glob.glob(directory)
print(jpgFiles)

# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/Cellar/tesseract/4.0.0/bin/tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'


# Simple image to string
for file in jpgFiles:
    print("Processing file in image convert: " + str(file))
    image = Image.open(file)
   # print(ImageSequence.Iterator(image)
    imageText = ""
    textFilename = str(file) + ".txt"
    for i, page in enumerate(ImageSequence.Iterator(image)):
        print("page", i)
        if os.path.isfile(textFilename):
            break
        else:
            imageText+=pytesseract.image_to_string(image)
        #print(imageText)
    if not os.path.isfile(textFilename): 
        #textFilename = "/txt/" + textFilename
        textFile = open(textFilename, "w") #make text file
        textFile.write(imageText) #write text to text file

In [ ]:
#writes all your text files to a dataframe

import glob
import errno
import xlsxwriter
import six
import io
from openpyxl import *
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from io import StringIO
import os

writer = pd.ExcelWriter('file.xlsx', engine='xlsxwriter')
    
wb = Workbook()
ws = wb.active


columns = ['Doc']
index=0
x=0


pathTXT = 'txt2/*.txt'
filesTXT = glob.glob(pathTXT)

raw_data = []
file_names = []
for name in filesTXT:
    try:
        with open(name, 'r') as myfile:
            print(name)
            data=myfile.read().replace('o\n', ' ')
            b = data.replace(' o ',  '')
            c = b.replace('*', "")
            d = c.replace('\n', ' ')
            e = d.replace('  ', ' ')
            f = e.replace('•', '')
            h = f.replace('--', '-')
            
            file_names.append(name)
            raw_data.append(h)
        column_cell = 'A'  
        #x+=1
        #index+=1
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
            
            
df = pd.DataFrame(raw_data, columns = ['Doc'])

#wb.save("ARRegs.xlsx")
            
#result_df = pd.read_excel('ARRegs.xlsx')
#print(df)
df.to_excel('ARRegulations2.xlsx', encoding='utf-8')
result_df = pd.read_excel('ARRegulations2.xlsx')
data_text=result_df['Doc']
data_text['index'] = data_text.index
documents = data_text
print(df)

            #word for line in open(words_file, 'r') for word in line.split()]

In [ ]:
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import *
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

nltk.download('wordnet')

In [ ]:
#uncomment the commented line down there and comment the line above it to lemmatize the documents
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
            #result.append(lemmatize_stemming(token))
    return result

In [ ]:
#preprocess the docs
processed_docs = df['Doc'].fillna('').astype(str).map(preprocess)
processed_docs = df['Doc'].map(preprocess)
processed_docs[:100]

In [ ]:
#create a 'dictionary' of all the words in the documents

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 100:
        break

In [ ]:
#filter out all the words that are in over 50% of the documents
dictionary.filter_extremes(no_below=1, no_above=0.5)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[:1]

In [ ]:
bow_doc_9 = bow_corpus[9]
for i in range(len(bow_doc_9)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_9[i][0], 
                                               dictionary[bow_doc_9[i][0]], 
bow_doc_9[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
#for doc in corpus_tfidf:
    #pprint(doc)
    #break

In [ ]:
#Change the number of topics you want with the 'num_topics' argument
#create your LDA model with this function
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=8, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=15, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
#model show
for index, score in sorted(lda_model[bow_corpus[9]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 30)))

In [ ]:
#tfidf model show
for index, score in sorted(lda_model_tfidf[bow_corpus[9]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 30)))

In [ ]:
#test
unseen_document = 'chemical and nuclear demilitarizations'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
top_score = 0
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    if score > top_score:
        print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))
        top_score = score

In [ ]:
#get the actual RN instead of the txt file name string
for i in range(len(file_names)):
    word = file_names[i].split('txt2/')
    word = word[1].split('.')
    file_names[i] = word[0]
    print(word[0])
    

In [ ]:
#get the number of pages in each pdf and output to an excel sheet
from io import StringIO
import os
import glob
import sys, getopt

directory = 'ARReg2/*.pdf' #path
pdfFiles = glob.glob(directory)
pagedf = pd.DataFrame(columns = ['AR', 'pages'])

pages = None
j=0
for doc in pdfFiles:
    #output = StringIO()
    #manager = PDFResourceManager()
    #converter = TextConverter(manager, output, laparams=LAParams())
    #interpreter = PDFPageInterpreter(manager, converter)
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)
    infile = open(doc, 'rb')
    i=0
    for page in PDFPage.get_pages(infile, pagenums):
        #interpreter.process_page(page)
        i+=1
    word = doc.split('ARReg2/')
    word = word[1].split('.')
    doc = word[0]
    #print(word[0])
    print(doc)
    pagedf.at[j, 'AR'] = doc
    print(i)
    pagedf.at[j, 'pages'] = i
    #infile.close()
    #converter.close()
    #manager.close()
    #output.close()
    j+=1
    #pagedf[:20]

pagedf.to_excel('output3.xlsx')

In [ ]:
#
import nltk
nltk.download('stopwords')
from nltk import BigramCollocationFinder
import os
#import pdf
import glob
import sys, getopt
from io import StringIO

#directory = 'txt/*.txt' #path
#words_file = glob.glob(directory)


#break up into individual words
#def read_words(words):
#    return [word for line in open(words, 'r') for word in line.split()]


#for file in words_file:
def ngram(ARstring, i):
    biGram = ""
    fourGram = ""
    nltk_tokens = nltk.word_tokenize(ARstring)
    ARBigrams = (list(nltk.bigrams(nltk_tokens)))
    ARWords = [word for word in nltk_tokens if word[0].isalpha()]
    ARText = nltk.Text(ARWords)
    
#create bigrams from our ARTokens
    ARBigramsFreqs = nltk.FreqDist(ARBigrams) # determine frequency of four-grams
    for words, count in ARBigramsFreqs.most_common(10):
        biGram += " ".join(list(words)) + "(" + str(count) + ")" + ", "
    #print(ARBigrams)
    
#create 4gram phrases  
    AR4grams = list(nltk.ngrams(ARWords, 4)) # create four-grams
    AR4gramsFreqs = nltk.FreqDist(AR4grams) # determine frequency of four-grams
    for words, count in AR4gramsFreqs.most_common(10): # for the 10 most common four-grams
        fourGram += " ".join(list(words)) + "(" + str(count) + ")" + ", "
        #final_df.at[i, 'Phrase_Count'] = count
        #ARText.collocations()
    final_df.at[i, 'Top_Bigrams'] = biGram
    final_df.at[i, 'Top_Phrases'] = fourGram

In [ ]:
#foreach processed_docs go through and compare assign a topic to each
#write the topic and the filename from processed doc to excel sheet
i=0
final_df = pd.DataFrame(columns = ['AR', 'LDA_Topic', 'Top_Bigrams', 'Top_Phrases'])
for doc in processed_docs:
    LDA = ""
    ARstring = ""
    for word in doc:
        ARstring += word + " "
    ngram(ARstring, i)
    bow_vector = dictionary.doc2bow(preprocess(ARstring))
    #print(ARstring)
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        #print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))
        LDA += lda_model.print_topic(index, 15)
    final_df.at[i, 'AR'] = file_names[i]
    final_df.at[i, 'LDA_Topic'] = LDA
    i+=1
)

In [ ]:
final_df[:20]

In [ ]:
final_df.to_excel('output4.xlsx')